In [1]:
import gc

import numpy as np
import pandas as pd
from random import sample

import hyperopt
from hyperopt import hp, Trials
from hyperopt.pyll.base import scope

from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from coremdlr.models import NetworkModel
from coremdlr.config import defaults

from coremdlr.datasets import WellLoader, FaciesDataset
from coremdlr.datasets.utils import infer_test_wells

import matplotlib.pyplot as plt
%matplotlib inline

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3

In [2]:
DEEPTEN_ARGS = {
    'feature' : 'image',
    'network' : 'deepten',
    'summary' : True,
    'sequence_size' : 12,
    'network_args' : {
        'backbone_cnn' : 'vgg16',
        'conv1x1' : 40,
        'encode_K' : 24,
        'apply_rowwise' : True,
        'lstm_features' : None,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
        'clipnorm' : 1.0
    }
}

IMAGE_ARGS = {
    'image_width' : 200,
    'crop_method' : 'center',
    'downsample' : 2
}


fit_args = {
    'batch_size' : 4,
    'epochs' : 25,
    'class_weighted' : True
}

CHKPT_FNAME = 'temp_best_weights.h5'

In [3]:
def pred_results_df(num_train, model_config):
    gc.collect()
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    pred_dfs = []
    
    for i, train_set in enumerate(train_sets):
        gc.collect()
        
        dset = FaciesDataset(train_set, infer_test_wells(train_set), features=['image'],
                            image_args=IMAGE_ARGS)
        
        dset.load_or_generate_data()
        
        model = NetworkModel(dset, model_args=model_config)
        
        chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_acc', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)

        earlystop_callback = EarlyStopping(monitor='val_acc', patience=3)
        
        model.fit(dset, callbacks=[chkpt_callback, earlystop_callback], verbose=False, **fit_args)
        
        model.network.load_weights(CHKPT_FNAME)
        
        #print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name)
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
        del dset; del model;
        
    return pd.concat(pred_dfs)

In [4]:
preds_df_3 = pred_results_df(3, DEEPTEN_ARGS)

[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('image', (19294, 32, 200, 3))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('image', (13006, 32, 200, 3))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('image', (10906, 32, 200, 3))]


/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Loading Well:  204-20a-7  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), ('image', (3105, 32, 200, 3))]
Loading Well:  204-19-7  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('image', (9807, 32, 200, 3))]
Loading Well:  204-19-6  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (1947,)), ('top', (1947,)), ('base', (1947,)), ('image', (1947, 32, 200, 3))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('image', (11210, 32, 200, 3))]
Loading Well:  204-20-1Z  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (1917,)), ('top', (1917,)), ('base', (1917,)), ('image', (1917, 32, 200, 3))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384, 200, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 384, 200, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 384, 200, 64) 36928       block1_conv1[0][0]               
_______________________________________________________

...

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Loading Well:  204-20a-7  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), ('image', (3105, 32, 200, 3))]
Loading Well:  204-19-7  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('image', (9807, 32, 200, 3))]
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('image', (19294, 32, 200, 3))]
Loading Well:  204-20-1Z  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (1917,)), ('top', (1917,)), ('base', (1917,)), ('image', (1917, 32, 200, 3))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('image', (3842, 32, 200, 3))]
Loading Well:  204-19-3A  from  /home/ross/Dropbox/core_data/facies/trai

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 384, 200, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 384, 200, 64) 1792        input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 384, 200, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 192, 100, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

694/695 [============================>.] - ETA: 0s - loss: 0.7365 - acc: 0.6187
Epoch 00002: val_acc improved from 0.28955 to 0.36215, saving model to temp_best_weights.h5
695/695 [==============================] - 134s 193ms/step - loss: 0.7367 - acc: 0.6187 - val_loss: 1.3716 - val_acc: 0.3622
Epoch 3/25
694/695 [============================>.] - ETA: 0s - loss: 0.6619 - acc: 0.6674
Epoch 00003: val_acc improved from 0.36215 to 0.53284, saving model to temp_best_weights.h5
695/695 [==============================] - 134s 193ms/step - loss: 0.6620 - acc: 0.6670 - val_loss: 1.3145 - val_acc: 0.5328
Epoch 4/25
694/695 [============================>.] - ETA: 0s - loss: 0.6020 - acc: 0.7054
Epoch 00004: val_acc did not improve from 0.53284
695/695 [==============================] - 134s 192ms/step - loss: 0.6035 - acc: 0.7045 - val_loss: 1.4842 - val_acc: 0.4188
Epoch 5/25
694/695 [============================>.] - ETA: 0s - loss: 0.5580 - acc: 0.7272
Epoch 00005: val_acc did not improve f

In [5]:
preds_df_3.to_csv('image_deepten_results_train_sets3_FINAL.csv')

In [6]:
accuracy_score(preds_df_3.y_pred.values, preds_df_3.y_true.values)

0.46419984208702364